In [1]:
# add to sys.path
import sys

sys.path.insert(0, "../")
sys.path.insert(0, "../src")

import time
import logging
import pprint
import pandas as pd
from datetime import datetime, timedelta, timezone
from dateutil.relativedelta import relativedelta
import plaid

# gcp
from google.cloud import bigquery

# SCHEMAS
from schemas.bq_table_schemas import BqTableSchemas
from schemas.cloud_schemas import CloudSchemas

# UTILS
from utils.bq_utils import BqUtils
from utils.google_cloud_utils import GcpUtils
from utils.plaid_utils import PlaidUtils
from utils.secrets_utils import SecretsUtils
from utils.crypto_utils import CryptoUtils
from utils.sendgrid_utils import SendgridUtils

# JOBS
from jobs.budget_values import BudgetValues
from jobs.financial_accounts import FinancialAccounts
from jobs.plaid_transactions import PlaidTransactions
from jobs.plaid_investments import PlaidInvestments
from jobs.data_quality_alerts import DataQualityAlerts

# get all secrets
sec = SecretsUtils()
secrets = sec.create_secrets_dict(plaid_secret_type="PROD")

PLAID_CLIENT_ID = secrets["PLAID_CLIENT_ID"]
PLAID_SECRET = secrets["PLAID_SECRET_PROD"]
PLAID_ACCESS_TOKENS = sec.get_access_token_secrets(secrets)
PLAID_HOST = plaid.Environment.Production
PLAID_PRODUCTS = ["liabilities", "transactions", "investments"]
PLAID_COUNTRY_CODES = ["US"]
CRYPTO_SECRETS = secrets["CRYPTO_SECRETS"]
SENDGRID_KEY = secrets["SENDGRID_KEY"]

# initialize clients
bq_client = bigquery.Client()
bq = BqUtils(bq_client=bq_client)
plaid_client = PlaidUtils(bq_client, PLAID_CLIENT_ID, PLAID_SECRET, PLAID_HOST)
financial_accounts = FinancialAccounts(bq_client, plaid_client)
plaid_transactions = PlaidTransactions(bq_client, plaid_client)
plaid_investments = PlaidInvestments(bq_client, plaid_client)
budget_values = BudgetValues(bq_client)
crypto = CryptoUtils()
bq_tables = BqTableSchemas()
gcp = GcpUtils(bq_client=bq_client)
sendgrid = SendgridUtils(SENDGRID_KEY)
dqa = DataQualityAlerts(bq_client)
cs = CloudSchemas()

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", 1000)

In [5]:
cs.financial_accounts().job_name

'financial-accounts-scheduler'

In [ ]:
schema = bq_tables.budget_values_YYYYMM()
schema.table_id

In [ ]:
df1 = pd.DataFrame({
    'A': [1, 2, 3],
    'B': [4, 5, 6]
})

df2 = pd.DataFrame({
    'X': [10, 20, 30],
    'Y': [40, 50, 60]
})

In [ ]:
message_1 = sendgrid.create_html_message_with_pandas_df("There is an issue with financial accounts", financial_accounts_df)
message_2 = sendgrid.create_html_message_with_pandas_df("Testing 2nd message", df2)

message_content = sendgrid.chain_html_messages([message_1, message_2])

email_message = sendgrid.construct_email_message(
    from_email="zach.s.cox@gmail.com",
    to_emails=["zach.s.cox+zsc-alerts@gmail.com"],
    email_subject="ZSC Alerts",
    html_message=message_content
)

sendgrid.send_email(email_message)

# sendgrid.get_email_content(email_message)

In [ ]:
import matplotlib.pyplot as plt


In [ ]:
df = bq.query("""
WITH
    account_values AS (
    SELECT 
        transaction_date,
        SUM(actual_amount) AS account_value,
    FROM `zsc-personal.personal_finance.personal_finance_tableau_20240815`
    WHERE metric_category = "ACCOUNTS"
    GROUP BY 1
    )
SELECT 
    transaction_date,
    account_value,
    ((account_value - LAG(account_value) OVER (ORDER BY transaction_date)) / ABS(LAG(account_value) OVER (ORDER BY transaction_date))) * 100 AS account_value_pct_chg
FROM account_values
ORDER BY transaction_date
""")

df.head()

In [ ]:

# Create the figure and the subplots
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8))

# First subplot - account_value
ax1.plot(df["transaction_date"], df["account_value"], color='blue')
ax1.set_title('Account Value Over Time')
ax1.set_xlabel('Transaction Date')
ax1.set_ylabel('Account Value')

# Second subplot - account_value_pct_chg
ax2.plot(df["transaction_date"], df["account_value_pct_chg"], color='green')
ax2.set_title('Account Value Percentage Change Over Time')
ax2.set_xlabel('Transaction Date')
ax2.set_ylabel('Account Value % Change')

# Adjust layout to prevent overlap
plt.tight_layout()

# Show the plot
plt.show()